In [1]:
import arcpy;
import os,sys;
import json,requests;
import datetime,http.client;
import zipfile,shutil,csv;

from ipywidgets import IntProgress;
from IPython.display import display;
import time;


### SWC Resources

Reusable function to verify and assign SWC resources to a map for use in processing


In [4]:
def swc_resources():
    
    rez = {};
    
    # Verify or Create Source filegeodatabase
    rez['source'] = os.getcwd() + os.sep + 'source.gdb';

    if not arcpy.Exists(rez['source']):
        print("  creating new source workspace");
        arcpy.CreateFileGDB_management(
             os.path.dirname(rez['source'])
            ,os.path.basename(rez['source'])
        );
        
    # Verify or Create Working filegeodatabase
    rez['working'] = os.getcwd() + os.sep + 'working.gdb';

    if not arcpy.Exists(rez['working']):
        print("  creating new working workspace");
        arcpy.CreateFileGDB_management(
             os.path.dirname(rez['working'])
            ,os.path.basename(rez['working'])
        );
        
    # Verify or Create Results filegeodatabase
    rez['results'] = os.getcwd() + os.sep + 'results.gdb';

    if not arcpy.Exists(rez['results']):
        print("  creating new results workspace");
        arcpy.CreateFileGDB_management(
             os.path.dirname(rez['results'])
            ,os.path.basename(rez['results'])
        );

    # Verify or Create qa directory
    rez['qa'] = os.getcwd() + os.sep + 'qa';

    if not arcpy.Exists(rez['qa']):
        print("  creating new qa directory");
        os.mkdir(rez['qa']);
        
    # Verify or Create resources directory
    rez['resources'] = os.getcwd() + os.sep + 'resources';

    if not arcpy.Exists(rez['resources']):
        print("  creating new resources directory");
        os.mkdir(rez['resources']);
        
    # Verify existence of files directory
    rez['files'] = os.getcwd() + os.sep + 'files';
    
    if not arcpy.Exists(rez['resources']):
        raise Exception('ERROR: project files directory not found');
        
    return rez;
        

### Scrape AGS 

Reusable function to scrape an ArcGIS Online resource into a local file geodatabase.
Note some online resources (namely Census) have additional download limits beyond the stated maxRecordCount.
Using a smaller forcelimit value will usually work around this.


In [5]:
def scrape_ags(host,path,fgdb,fc,forcelimit=None):
    
    if arcpy.Exists(fgdb + os.sep + fc):
        arcpy.Delete_management(fgdb + os.sep + fc);
        
    headers = {"Content-type": "application/x-www-form-urlencoded", "Accept": "text/plain"};
    conn = http.client.HTTPSConnection(host);
    conn.request("POST",path,"f=json",headers);
    response = conn.getresponse();
    data = response.read();
    json_data = json.loads(data);
    if not 'currentVersion' in json_data:
        raise ValueError("Error, unable to query https://" + host + path);
    extraction_amount = json_data['maxRecordCount'];
    if forcelimit is not None and forcelimit < extraction_amount:
        extraction_amount = forcelimit;
    where = "1=1";
    params = "where={}&returnIdsOnly=true&returnGeometry=false&f=json".format(where);
    conn = http.client.HTTPSConnection(host);
    conn.request("POST",path + "/query",params,headers);
    response = conn.getresponse();
    data = response.read();
    json_data = json.loads(data);
    ary_oid   = sorted(json_data['objectIds']);
    oid_name  = json_data['objectIdFieldName'];
    oid_count = len(ary_oid);
    
    initial_hit = True;
    counter = 0;
    while counter <= oid_count - 1:
        if counter + extraction_amount > oid_count - 1:
            int_max = oid_count - 1;
        else:
            int_max = counter + extraction_amount - 1;
        where = oid_name + ' >= ' + str(ary_oid[counter]) + ' AND ' + oid_name + ' <= ' + str(ary_oid[int_max]);
        print("  pulling records where " + where);
        fields = "*";
        params = "where={}&outFields={}&returnGeometry=true&outSR=4269&f=json".format(where, fields);
        conn = http.client.HTTPSConnection(host);
        conn.request("POST",path + "/query",params,headers);
        response = conn.getresponse();
        data = response.read(); 
        json_data = json.loads(data);
        ef = arcpy.AsShape(json_data,True);
        if initial_hit:
            arcpy.management.CopyFeatures(ef,fgdb + os.sep + fc)
            initial_hit = False;
        else:
            arcpy.Append_management(ef,fgdb + os.sep + fc,"NO_TEST");
        counter += extraction_amount;
        
    conn.close(); 
    del conn;
    print("  Scrape complete.");
    return True;


### Tab Downloaders

Reusable functions to download and import csv or tab delimited text into local filegeodatabase 
using optional custom fieldmappings

In [ ]:
def downloadtab(url,filename):
    if arcpy.Exists(filename):
        arcpy.Delete_management(filename);
    print("  downloading file");
    with open(filename,'wb') as f,requests.get(url,stream=True) as r:
        for line in r.iter_lines():
            f.write(line + '\n'.encode());
    return True;
    
def tab2fc(filename,fgdb,fc,longname,latname,field_mapping=None):
    
    if arcpy.Exists('memory' + os.sep + 'tempTable'):
        arcpy.Delete_management('memory' + os.sep + 'tempTable');
  
    print("  loading to table");
    arcpy.TableToTable_conversion(
         in_rows       = filename
        ,out_path      = 'memory'
        ,out_name      = 'tempTable'
        ,field_mapping = field_mapping
    );
    
    if arcpy.Exists(fgdb + os.sep + fc):
        arcpy.Delete_management(fgdb + os.sep + fc);
        
    print("  converting to NAD83 points");
    arcpy.management.XYTableToPoint(
         in_table          = 'memory' + os.sep + 'tempTable'
        ,out_feature_class = fgdb + os.sep + fc
        ,x_field           = longname
        ,y_field           = latname
        ,coordinate_system = arcpy.SpatialReference(4269)
    );
    
    arcpy.Delete_management('memory' + os.sep + 'tempTable');
    return True;

def tab2tab(filename,fgdb,fc,field_mapping=None):
    
    if arcpy.Exists(fgdb + os.sep + fc):
        arcpy.Delete_management(fgdb + os.sep + fc);
    
    print("  loading to table");
    arcpy.TableToTable_conversion(
         in_rows       = filename
        ,out_path      = fgdb
        ,out_name      = fc
        ,field_mapping = field_mapping
    );
        
    return True;

def fmtext(infc,fieldname,fieldlength):
    fm = arcpy.FieldMap();
    fm.addInputField(infc,fieldname);
    nf = fm.outputField;
    nf.type = 'Text';
    nf.length = fieldlength;
    fm.outputField = nf;
    return fm;

def fmint(infc,fieldname):
    fm = arcpy.FieldMap();
    fm.addInputField(infc,fieldname);
    nf = fm.outputField;
    nf.type = 'Integer';
    fm.outputField = nf;
    return fm;

def fmdouble(infc,fieldname):
    fm = arcpy.FieldMap();
    fm.addInputField(infc,fieldname);
    nf = fm.outputField;
    nf.type = 'Double';
    fm.outputField = nf;
    return fm;
    

### TEMP and PRECIP Table Makers

Functions to create empty TEMP and PRECIP tables

In [ ]:
def createTEMP(fgdb,fc):
    
    if arcpy.Exists(fgdb + os.sep + fc):
        arcpy.Delete_management(fgdb + os.sep + fc);
    
    arcpy.CreateFeatureclass_management(
         out_path      = fgdb
        ,out_name      = fc
        ,geometry_type = "POINT"
        ,has_m         = "DISABLED"
        ,has_z         = "DISABLED"
        ,spatial_reference = arcpy.SpatialReference(4269) 
    );
    
    arcpy.management.AddFields(
         in_table = fgdb + os.sep + fc
        ,field_description = [
             ['StationId','TEXT'  ,'StationId',14]
            ,['Jan'      ,'DOUBLE','Jan']
            ,['Feb'      ,'DOUBLE','Feb']
            ,['Mar'      ,'DOUBLE','Mar']
            ,['Apr'      ,'DOUBLE','Apr']
            ,['May'      ,'DOUBLE','May']
            ,['Jun'      ,'DOUBLE','Jun']
            ,['Jul'      ,'DOUBLE','Jul']
            ,['Aug'      ,'DOUBLE','Aug']
            ,['Sep'      ,'DOUBLE','Sep']
            ,['Oct'      ,'DOUBLE','Oct']
            ,['Nov'      ,'DOUBLE','Nov']
            ,['Dec'      ,'DOUBLE','Dec']
            ,['Ann'      ,'DOUBLE','Ann']
         ]
    );
    
def createPREC(fgdb,fc):
    
    if arcpy.Exists(fgdb + os.sep + fc):
        arcpy.Delete_management(fgdb + os.sep + fc);
    
    arcpy.CreateFeatureclass_management(
         out_path      = fgdb
        ,out_name      = fc
        ,geometry_type = "POINT"
        ,has_m         = "DISABLED"
        ,has_z         = "DISABLED"
        ,spatial_reference = arcpy.SpatialReference(4269) 
    );
    
    arcpy.management.AddFields(
         in_table = fgdb + os.sep + fc
        ,field_description = [
             ['StationId','TEXT'  ,'StationId',14]
            ,['Jan'      ,'DOUBLE','Jan']
            ,['Feb'      ,'DOUBLE','Feb']
            ,['Mar'      ,'DOUBLE','Mar']
            ,['Apr'      ,'DOUBLE','Apr']
            ,['May'      ,'DOUBLE','May']
            ,['Jun'      ,'DOUBLE','Jun']
            ,['Jul'      ,'DOUBLE','Jul']
            ,['Aug'      ,'DOUBLE','Aug']
            ,['Sep'      ,'DOUBLE','Sep']
            ,['Oct'      ,'DOUBLE','Oct']
            ,['Nov'      ,'DOUBLE','Nov']
            ,['Dec'      ,'DOUBLE','Dec']
            ,['Ann'      ,'DOUBLE','Ann']
         ]
    );
    
def createGEVdepth(fgdb,fc):
    
    if arcpy.Exists(fgdb + os.sep + fc):
        arcpy.Delete_management(fgdb + os.sep + fc);
    
    arcpy.CreateFeatureclass_management(
         out_path      = fgdb
        ,out_name      = fc
        ,geometry_type = "POINT"
        ,has_m         = "DISABLED"
        ,has_z         = "DISABLED"
        ,spatial_reference = arcpy.SpatialReference(4269) 
    );
    
    arcpy.management.AddFields(
         in_table = fgdb + os.sep + fc
        ,field_description = [
             ['StationId'   ,'TEXT'  ,'StationId',14]
            ,['z5yrann_mm'  ,'DOUBLE','5yrann_mm']
            ,['z10yrann_mm' ,'DOUBLE','10yrann_mm']
            ,['z15yrann_mm' ,'DOUBLE','15yrann_mm']
            ,['z30yrann_mm' ,'DOUBLE','30yrann_mm']
            ,['z50yrann_mm' ,'DOUBLE','50yrann_mm']
            ,['z100yrann_mm','DOUBLE','100yrann_mm']
         ]
    );
    

In [ ]:
def tabWriter(fc,outfile,flds,outflds=None):

    if outflds is None:
        outflds = flds;
        
    with open(outfile,'w',newline='',encoding='utf-8') as f:
        writer = csv.writer(f,delimiter='\t',lineterminator='\n');

        with arcpy.da.SearchCursor(fc,flds) as incur:
            writer.writerow(outflds);

            for row in incur:
                if row[0] is not None:
                    writer.writerow(row);

    cnt = arcpy.GetCount_management(outfile)[0];         
    print("  wrote " + str(cnt) + " records to " + os.path.basename(outfile) + ".");
